<a href="https://colab.research.google.com/github/Leandro-Bernardo/Neural-Network-for-fire-detection/blob/main/1_Rede_Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 27 epochs, learning rate 0.001 ,  4 layers de convolução  (16,32,32,32)

#### Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Pré processamento das Imagens

In [ ]:
import os

In [ ]:
local = input('Qual Treinamento?')

Qual Treinamento?/content/drive/MyDrive/TCC/Videos/Treinamento Original


In [ ]:
dir = list(os.listdir(local))
dir

['Fumaca_Original', 'Nao_Fumaca_Original']

In [ ]:
print(f'{dir[0]} contem :', len(os.listdir(f'{local}/{dir[0]}')),'fotos')
print(f'{dir[1]} contem :', len(os.listdir(f'{local}/{dir[1]}')),'fotos')

size = sum([len(os.listdir(f'{local}/{dir[0]}')) ,len(os.listdir(f'{local}/{dir[1]}'))])
print('Total:', size)

Fumaca_Original contem : 93 fotos
Nao_Fumaca_Original contem : 131 fotos
Total: 224


In [ ]:
amostras = int(size/8)
amostras


28

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Para reescalar as imagens em 1./255  (normalizar)
reescala = ImageDataGenerator(rescale=1/255)

# Flow training images in batches using train_datagen generator
imagens_treinamento = reescala.flow_from_directory(
        local,  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size= amostras,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 224 images belonging to 2 classes.


#### Rede Neural

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
rede = tf.keras.models.Sequential([
    
    # tf.keras.layers.Conv2D ( numero de filtros , matriz n x n, função de ativação, dados de entrada)
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)), # input_shape(linhas, colunas, camadas (de cor))
    tf.keras.layers.MaxPooling2D(2, 2),  # max pool usando uma matriz n x n
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Flatten os resultados para usar na Rede Neural Densa
    tf.keras.layers.Flatten(),

    # tf.keras.layers.Dense( N, função) cria uma Hidden Layer com N neuronios
    tf.keras.layers.Dense(256, activation='relu'),

    # Só um neurônio de saída por causa da função de ativação (Sigmoid). Esse neurônio recebe valor 0 ou 1 (0 para classe 1 e 1 para a outra classe)  
    tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
rede.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 32)       0

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
rede.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

#### Treinamento

In [ ]:
#para trocar o diretorio 
%cd '/content/drive/MyDrive/TCC/Videos/' 

/content/drive/MyDrive/TCC/Videos


In [ ]:
history = rede.fit(
      imagens_treinamento,
      steps_per_epoch=8,  # total de x imagens e cada batch com y imagens , logo o passo será x/y 
      epochs=27,     
      verbose=1)

Epoch 1/27
8/8 [==============================] - 26s 3s/step - loss: 0.6870 - accuracy: 0.5848
Epoch 2/27
8/8 [==============================] - 17s 2s/step - loss: 0.6830 - accuracy: 0.5848
Epoch 3/27
8/8 [==============================] - 17s 2s/step - loss: 0.6679 - accuracy: 0.5848
Epoch 4/27
8/8 [==============================] - 17s 2s/step - loss: 0.6731 - accuracy: 0.5804
Epoch 5/27
8/8 [==============================] - 17s 2s/step - loss: 0.6945 - accuracy: 0.7188
Epoch 6/27
8/8 [==============================] - 17s 2s/step - loss: 0.6223 - accuracy: 0.6027
Epoch 7/27
8/8 [==============================] - 17s 2s/step - loss: 0.5623 - accuracy: 0.7455
Epoch 8/27
8/8 [==============================] - 17s 2s/step - loss: 0.4756 - accuracy: 0.7812
Epoch 9/27
8/8 [==============================] - 17s 2s/step - loss: 0.6755 - accuracy: 0.8929
Epoch 10/27
8/8 [==============================] - 17s 2s/step - loss: 0.3224 - accuracy: 0.9375
Epoch 11/27
8/8 [======================

#### Validamento

In [ ]:
%cd '/content/drive/MyDrive/TCC/Videos/Validação'

/content/drive/MyDrive/TCC/Videos/Validação


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/drive/MyDrive/TCC/Videos/Validação/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = rede.predict(images, batch_size=10)
  print(classes[0])
    
  if classes[0]>0.5:
    print(fn + " não é fumaça")
  else:
    print(fn + " é fumaça ")

Saving corte_68.jpeg to corte_68.jpeg
[0.59939224]
corte_68.jpeg não é fumaça
